<a href="https://colab.research.google.com/github/johrosa/ls-FloodMonitoring/blob/main/country_scale_flood_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Flood Detection
This Python notebook presents a comprehensive workflow aimed at detecting country-wide flood events through the utilization of radar imagery as described in:

*Johary, R.; Révillion, C.; Catry, T.; Alexandre, C.; Mouquet, P.; Rakotoniaina, S.; Pennober, G.; Rakotondraompiana, S. Detection of Large-Scale Floods Using Google Earth Engine and Google Colab. Remote Sens. 2023, 15, 5368. https://doi.org/10.3390/rs15225368*

Some parameters are required:

- `PROJECT`: Earth engine cloud project, If not available, register new one in https://code.earthengine.google.com/register

- `COUNTRY`: Name of the country in the [FAO GAUL](https://en.wikipedia.org/wiki/Global_Administrative_Unit_Layers) Database where to perform the analysis.
- `EVENT`: Name of the flood event.
- `START_DATE`: The earliest date to include images for (inclusive).
- `END_DATE`: The latest date to include images for (exclusive).
- `POLARIZATION`: The Sentinel-1 image polarization to select for processing.
- `ORBIT_PASS`: The orbits to consider (string: ASCENDING or DESCENDING).
- `SPECKLE_FILTER`: Type of speckle filtering to apply:
  - 'BOXCAR' - Applies a boxcar filter.
  - 'LEE' - Applies a Lee filter based on [1].
  - 'GAMMA MAP' - Applies a Gamma maximum a-posterior speckle on the image based on [2] & [3].
  - 'REFINED LEE' - Applies the Refined Lee speckle filter based on [4].
  - 'LEE SIGMA' - Applies the improved Lee sigma speckle filter based on [5].
- `KERNEL_SIZE`: Size of the filter spatial window applied in speckle filtering. It must be a positive odd integer.
- `REFERENCE`: The method for selecting reference images:
  - DRY: Reference images taken during the dry season.
  - BEFORE: Reference images taken just before the event.
  - MULTI: All available images taken in previous years and during the same period as the event.
- `ASSET_PATH`: Earth Engine asset path where to store results. (e.g., "users/username/flood_detection_results")
- `SCALE`: Spatial resolution of output images in meters.


In [ ]:
#@title Parameters setting { run: "auto", display-mode: "form" }

PROJECT = "ee-rnrimpact" #@param {type: "string"}
#  <font size="1"> &nbsp;&nbsp;&nbsp; Earth engine cloud project, If not available, register new one in https://code.earthengine.google.com/register </font>
COUNTRY = "Madagascar" #@param {type:"string"}
EVENT = 'Gemane' #@param {type:"string"}
START_DATE = '2024-03-28' #@param {type:"date"}
END_DATE = '2024-03-31' #@param {type:"date"}

POLARISATION = 'VVVH' #@param ["VV","VH", "VVVH"]
ORBIT_PASS = "DESCENDING" #@param ["ASCENDING","DESCENDING"]

FILTER = "REFINED LEE" #@param ['BOXCAR', 'GAMMA MAP', 'LEE', 'REFINED LEE', 'LEE SIGMA']
KERNEL_SIZE = 3 #@param {type:"number"}

REFERENCE = "MULTI" #@param ["MULTI", "BEFORE", "DRY"]

ASSET_PATH = 'projects/ee-rnrimpact/assets' #@param {type:"string"}
SCALE = 20 #@param {type:"number"}

match POLARISATION:
    case "VV":
         POLARISATION = ['VV']
    case "VH":
         POLARISATION = ['VH']
    case "VVVH":
         POLARISATION = ['VV','VH']


In [ ]:
#@title Import, authenticate and initialize the Earth Engine library
import ee
ee.Authenticate()
ee.Initialize(project=PROJECT)


----

##Speckle filter


In [ ]:
#@title function for Speckle Filtering

def spkFilter(image, filterName="GAMMA MAP", kernel_size=3):

  """
  Function to apply speckle filtering on ee.Image object type
  Modified from :
  Mullissa, A., Vollrath, A., Odongo-Braun, C., Slagter, B., Balling, J., Gou, Y., Gorelick, N., Reiche, J., 2021.
    Sentinel-1 SAR Backscatter Analysis Ready Data Preparation in Google Earth Engine. Remote Sensing 13, 1954.
    https://doi.org/10.3390/rs13101954
  """


  def boxcar(image, kernel_size):

    bandNames = image.bandNames().remove('angle')
    # Define a boxcar kernel
    kernel = ee.Kernel.square(**{'radius': (KERNEL_SIZE/2), 'units': 'pixels', 'normalize': True})
    # Apply boxcar
    output = image.select(bandNames).convolve(kernel).rename(bandNames);
    return image.addBands(output, None, True)


  # GammaMap filter for ee.Image
  def gammamap(image,kernel_size):

    import math
    enl = 5
    bandNames = image.bandNames().remove('angle')
    #Neighbourhood stats
    reducers = ee.Reducer.mean().combine(**{'reducer2': ee.Reducer.stdDev(), 'sharedInputs': True})
    stats = image.select(bandNames).reduceNeighborhood(**{'reducer': reducers,\
                                                          'kernel': ee.Kernel.square(kernel_size/2,'pixels'),\
                                                          'optimization': 'window'})
    meanBand = bandNames.map(lambda bandName: ee.String(bandName).cat('_mean'))
    stdDevBand = bandNames.map(lambda bandName: ee.String(bandName).cat('_stdDev'))
    z = stats.select(meanBand)
    sigz = stats.select(stdDevBand)

    #local observed coefficient of variation
    ci = sigz.divide(z)

    #noise coefficient of variation (or noise sigma)
    cu = 1.0/math.sqrt(enl)

    #threshold for the observed coefficient of variation
    cmax = math.sqrt(2.0) * cu
    cu = ee.Image.constant(cu)
    cmax = ee.Image.constant(cmax)
    enlImg = ee.Image.constant(enl)
    oneImg = ee.Image.constant(1)
    twoImg = ee.Image.constant(2)
    alpha = oneImg.add(cu.pow(2)).divide(ci.pow(2).subtract(cu.pow(2)))



    #Implements the Gamma MAP filter described in equation 11 in Lopez et al. 1990
    q = image.select(bandNames).expression("z**2 * (z * alpha - enl - 1)**2 + 4 * alpha * enl * b() * z", {'z': z, 'alpha': alpha,'enl': enl})
    rHat = z.multiply(alpha.subtract(enlImg).subtract(oneImg)).add(q.sqrt()).divide(twoImg.multiply(alpha))


    #if ci <= cu then its a homogenous region ->> boxcar filter
    zHat = (z.updateMask(ci.lte(cu))).rename(bandNames)

    #if cmax > ci > cu then its a textured medium ->> apply Gamma MAP filter
    rHat = (rHat.updateMask(ci.gt(cu)).updateMask(ci.lt(cmax))).rename(bandNames)

    #if ci>=cmax then its strong signal ->> retain
    x = image.select(bandNames).updateMask(ci.gte(cmax)).rename(bandNames)

    # Merge
    output = ee.ImageCollection([zHat,rHat,x]).sum()
    return image.addBands(output, None, True)


  def leefilter(image,kernel_size):

    import math
    bandNames = image.bandNames().remove('angle')
    #S1-GRD images are multilooked 5 times in range
    enl = 5
    #Compute the speckle standard deviation
    eta = 1.0/math.sqrt(enl)
    eta = ee.Image.constant(eta)
    #MMSE estimator
    #Neighbourhood mean and variance
    oneImg = ee.Image.constant(1)

    reducers = ee.Reducer.mean().combine(**{\
                        'reducer2': ee.Reducer.variance(),\
                        'sharedInputs': True\
                        })

    stats = image.select(bandNames).reduceNeighborhood(**{'reducer': reducers,\
                                                          'kernel': ee.Kernel.square(kernel_size/2,'pixels'),\
                                                          'optimization': 'window'})
    meanBand = bandNames.map(lambda bandName: ee.String(bandName).cat('_mean'))
    varBand = bandNames.map(lambda bandName: ee.String(bandName).cat('_variance'))

    z_bar = stats.select(meanBand)
    varz = stats.select(varBand)

    #Estimate weight
    varx = (varz.subtract(z_bar.pow(2).multiply(eta.pow(2)))).divide(oneImg.add(eta.pow(2)))
    b = varx.divide(varz)

    #if b is negative set it to zero
    new_b = b.where(b.lt(0), 0);
    output = oneImg.subtract(new_b).multiply(z_bar.abs()).add(new_b.multiply(image.select(bandNames)))
    output = output.rename(bandNames)
    return image.addBands(output, None, True)


  #refined lee filter for ee.Image
  def refinedLee(image):
    bandNames = image.bandNames().remove('angle')
    def filterForBands(bn):
      img = image.select([bn]).resample('bilinear')

      #img must be linear, i.e. not in dB!
      #Set up 3x3 kernels
      weights3 = ee.List.repeat(ee.List.repeat(1,3),3)
      kernel3 = ee.Kernel.fixed(3,3, weights3, 1, 1, False)

      mean3 = img.reduceNeighborhood(ee.Reducer.mean(), kernel3)
      variance3 = img.reduceNeighborhood(ee.Reducer.variance(), kernel3)

      #Use a sample of the 3x3 windows inside a 7x7 windows to determine gradients and directions
      sample_weights = ee.List([[0,0,0,0,0,0,0], [0,1,0,1,0,1,0],[0,0,0,0,0,0,0], [0,1,0,1,0,1,0], [0,0,0,0,0,0,0], [0,1,0,1,0,1,0],[0,0,0,0,0,0,0]])

      sample_kernel = ee.Kernel.fixed(7,7, sample_weights, 3,3, False)

      #Calculate mean and variance for the sampled windows and store as 9 bands
      sample_mean = mean3.neighborhoodToBands(sample_kernel)
      sample_var = variance3.neighborhoodToBands(sample_kernel)

      #Determine the 4 gradients for the sampled windows
      gradients = sample_mean.select(1).subtract(sample_mean.select(7)).abs()
      gradients = gradients.addBands(sample_mean.select(6).subtract(sample_mean.select(2)).abs())
      gradients = gradients.addBands(sample_mean.select(3).subtract(sample_mean.select(5)).abs())
      gradients = gradients.addBands(sample_mean.select(0).subtract(sample_mean.select(8)).abs())

      #And find the maximum gradient amongst gradient bands
      max_gradient = gradients.reduce(ee.Reducer.max())

      #Create a mask for band pixels that are the maximum gradient
      gradmask = gradients.eq(max_gradient)

      #Duplicate gradmask bands: each gradient represents 2 directions
      gradmask = gradmask.addBands(gradmask)

      #Determine the 8 directions
      directions = sample_mean.select(1).subtract(sample_mean.select(4)).gt(sample_mean.select(4).subtract(sample_mean.select(7))).multiply(1)
      directions = directions.addBands(sample_mean.select(6).subtract(sample_mean.select(4)).gt(sample_mean.select(4).subtract(sample_mean.select(2))).multiply(2))
      directions = directions.addBands(sample_mean.select(3).subtract(sample_mean.select(4)).gt(sample_mean.select(4).subtract(sample_mean.select(5))).multiply(3))
      directions = directions.addBands(sample_mean.select(0).subtract(sample_mean.select(4)).gt(sample_mean.select(4).subtract(sample_mean.select(8))).multiply(4))

      #The next 4 are the not() of the previous 4
      directions = directions.addBands(directions.select(0).Not().multiply(5))
      directions = directions.addBands(directions.select(1).Not().multiply(6))
      directions = directions.addBands(directions.select(2).Not().multiply(7))
      directions = directions.addBands(directions.select(3).Not().multiply(8))

      #Mask all values that are not 1-8
      directions = directions.updateMask(gradmask)

      # "collapse" the stack into a singe band image (due to masking, each pixel has just one value (1-8) in it's directional band, and is otherwise masked)
      directions = directions.reduce(ee.Reducer.sum())

      sample_stats = sample_var.divide(sample_mean.multiply(sample_mean))

      #Calculate localNoiseVariance
      sigmaV = sample_stats.toArray().arraySort().arraySlice(0,0,5).arrayReduce(ee.Reducer.mean(), [0])

      #Set up the 7*7 kernels for directional statistics
      rect_weights = ee.List.repeat(ee.List.repeat(0,7),3).cat(ee.List.repeat(ee.List.repeat(1,7),4))

      diag_weights = ee.List([[1,0,0,0,0,0,0], [1,1,0,0,0,0,0], [1,1,1,0,0,0,0], [1,1,1,1,0,0,0], [1,1,1,1,1,0,0], [1,1,1,1,1,1,0], [1,1,1,1,1,1,1]])

      rect_kernel = ee.Kernel.fixed(7,7, rect_weights, 3, 3, False)
      diag_kernel = ee.Kernel.fixed(7,7, diag_weights, 3, 3, False)

      #Create stacks for mean and variance using the original kernels. Mask with relevant direction.
      dir_mean = img.reduceNeighborhood(ee.Reducer.mean(), rect_kernel).updateMask(directions.eq(1))
      dir_var = img.reduceNeighborhood(ee.Reducer.variance(), rect_kernel).updateMask(directions.eq(1))

      dir_mean = dir_mean.addBands(img.reduceNeighborhood(ee.Reducer.mean(), diag_kernel).updateMask(directions.eq(2)))
      dir_var = dir_var.addBands(img.reduceNeighborhood(ee.Reducer.variance(), diag_kernel).updateMask(directions.eq(2)))

      #and add the bands for rotated kernels
      for i in range(1,5):
        dir_mean = dir_mean.addBands(img.reduceNeighborhood(ee.Reducer.mean(), rect_kernel.rotate(i)).updateMask(directions.eq(2*i+1)))
        dir_var = dir_var.addBands(img.reduceNeighborhood(ee.Reducer.variance(), rect_kernel.rotate(i)).updateMask(directions.eq(2*i+1)))
        dir_mean = dir_mean.addBands(img.reduceNeighborhood(ee.Reducer.mean(), diag_kernel.rotate(i)).updateMask(directions.eq(2*i+2)))
        dir_var = dir_var.addBands(img.reduceNeighborhood(ee.Reducer.variance(), diag_kernel.rotate(i)).updateMask(directions.eq(2*i+2)))

      #"collapse" the stack into a single band image (due to masking, each pixel has just one value in it's directional band, and is otherwise masked)
      dir_mean = dir_mean.reduce(ee.Reducer.sum())
      dir_var = dir_var.reduce(ee.Reducer.sum())
      varX = dir_var.subtract(dir_mean.multiply(dir_mean).multiply(sigmaV)).divide(sigmaV.add(1.0))

      #A finally generate the filtered value
      b = varX.divide(dir_var)
      return dir_mean.add(b.multiply(img.subtract(dir_mean)))\
        .arrayProject([0])\
        .arrayFlatten([['sum']])\
        .float()

    #apply filter for each band
    result = ee.ImageCollection(bandNames.map(filterForBands)).toBands().rename(bandNames).copyProperties(image)


    return image.addBands(result, None, True)


  def leesigma(image,kernel_size):

    import math
    #parameters
    Tk = ee.Image.constant(7)

    #number of bright pixels in a 3x3 window
    sigma = 0.9
    enl = 4
    target_kernel = 3
    bandNames = image.bandNames().remove('angle')
    #compute the 98 percentile intensity
    z98 = image.select(bandNames).reduceRegion(**{
                                                  'reducer': ee.Reducer.percentile([98]),
                                                  'geometry': image.geometry(),
                                                  'scale':10,
                                                  'maxPixels':1e13
                                                  }).toImage()
    #select the strong scatterers to retain
    brightPixel = image.select(bandNames).gte(z98)
    K = brightPixel.reduceNeighborhood(**{'reducer': ee.Reducer.countDistinctNonNull(),
                              'kernel': ee.Kernel.square((target_kernel/2) ,'pixels')});
    retainPixel = K.gte(Tk)
    #compute the a-priori mean within a 3x3 local window
    #original noise standard deviation
    eta = 1.0/math.sqrt(enl)
    eta = ee.Image.constant(eta)
    #MMSE applied to estimate the a-priori mean
    reducers = ee.Reducer.mean().combine(**{
                        'reducer2': ee.Reducer.variance(),
                        'sharedInputs': True
                        })
    stats = image.select(bandNames).reduceNeighborhood(**{'reducer': reducers,\
                                                          'kernel': ee.Kernel.square(target_kernel/2,'pixels'),
                                                          'optimization': 'window'})
    meanBand = bandNames.map(lambda bandName: ee.String(bandName).cat('_mean'))
    varBand = bandNames.map(lambda bandName: ee.String(bandName).cat('_variance'))
    z_bar = stats.select(meanBand)
    varz = stats.select(varBand)

    oneImg = ee.Image.constant(1)
    varx = (varz.subtract(z_bar.abs().pow(2).multiply(eta.pow(2)))).divide(oneImg.add(eta.pow(2)))
    b = varx.divide(varz)
    xTilde = oneImg.subtract(b).multiply(z_bar.abs()).add(b.multiply(image.select(bandNames)));

    #Compute the sigma range
    #Lookup table (J.S.Lee et al 2009) for range and eta values for intensity (only 4 look is shown here)
    LUT = ee.Dictionary({'0.5': ee.Dictionary({'I1': 0.694,'I2': 1.385,'eta': 0.1921}),
                                  '0.6': ee.Dictionary({'I1': 0.630,'I2': 1.495,'eta': 0.2348}),
                                  '0.7': ee.Dictionary({'I1': 0.560,'I2': 1.627,'eta': 0.2825}),
                                  '0.8': ee.Dictionary({'I1': 0.480,'I2': 1.804,'eta': 0.3354}),
                                  '0.9': ee.Dictionary({'I1': 0.378,'I2': 2.094,'eta': 0.3991}),
                                  '0.95': ee.Dictionary({'I1': 0.302,'I2': 2.360,'eta': 0.4391})})

    #extract data from lookup
    sigmaImage = ee.Dictionary(LUT.get(str(sigma))).toImage()
    I1 = sigmaImage.select('I1')
    I2 = sigmaImage.select('I2')
    #new speckle sigma
    nEta = sigmaImage.select('eta')
    #establish the sigma ranges
    I1 = I1.multiply(xTilde)
    I2 = I2.multiply(xTilde)

    #apply the minimum mean square error (MMSE) filter for pixels in the sigma range
    #MMSE estimator
    mask = image.select(bandNames).gte(I1).Or(image.select(bandNames).lte(I2))
    z = image.select(bandNames).updateMask(mask)

    stats = z.reduceNeighborhood(**{'reducer': reducers,'kernel': ee.Kernel.square(kernel_size/2,'pixels'), 'optimization': 'window'})

    z_bar = stats.select(meanBand)
    varz = stats.select(varBand)

    varx = (varz.subtract(z_bar.abs().pow(2).multiply(nEta.pow(2)))).divide(oneImg.add(nEta.pow(2)))
    b = varx.divide(varz)
    #if b is negative set it to zero
    new_b = b.where(b.lt(0), 0);
    xHat = oneImg.subtract(new_b).multiply(z_bar.abs()).add(new_b.multiply(z))

    #remove the applied masks and merge the retained pixels and the filtered pixels
    xHat = image.select(bandNames).updateMask(retainPixel).unmask(xHat)
    output = ee.Image(xHat).rename(bandNames)
    return image.addBands(output, None, True)

  if filterName == "BOXCAR":
    return boxcar(image, kernel_size)
  elif filterName == "GAMMA MAP":
    return gammamap(image, kernel_size)
  elif  filterName == "LEE":
    return leefilter(image, kernel_size)
  elif  filterName == "REFINED LEE":
    return refinedLee(image)
  elif  filterName == "LEE SIGMA":
    return leesigma(image, kernel_size)
  else:
    raise ValueError(f"'{filterName}' filter unknown, must be one of 'BOXCAR', 'GAMMA MAP', 'LEE', 'REFINED LEE' or 'LEE SIGMA'")


##NDR

In [ ]:
#@title function for flood detection using NDR method and several reference images
def ndr(image, ref):
    band_names = image.bandNames().remove('angle')
    image = image.select(band_names)
    ref = ref.select(band_names)
    th = ee.Dictionary({'VV':-0.35, 'VH':-0.35})
    ndr = band_names.map(lambda bn: image.select([bn]).subtract(ref.select([bn]))
                        .divide(image.select([bn]).add(ref.select([bn]))).lt(th.getNumber(bn)).rename([bn]))

    return ee.ImageCollection(ndr).toBands().reduce(ee.Reducer.sum()).gt(0).toUint8()


def ndr_autoRef(image, refMode = "MULTI"):

  def cleanIsolatedPixel(image):

      connectedCount = image.unmask().connectedPixelCount(15, True)
      image_cleaned = image.where(connectedCount.lte(10), image.Not())
      return image_cleaned


  band_names = image.bandNames().remove('angle')
  image = image.select(band_names)
  date = image.date()

  #load HAND image
  hand = ee.Image('users/gena/GlobalHAND/30m/hand-1000')
  handTh = hand.lt(5)

  if refMode == "MULTI":
    # find images with same configurations and acquired before detection period
    refCol = ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')\
            .filterBounds(image.geometry().centroid())\
            .filterDate('2014-01-01', date.format("yyyy-MM-dd"))\
            .filter(ee.Filter.calendarRange(
                # retain only images acquired in the same periode of the year as the image
                date.advance(-12, 'day').getRelative("day","year"),
                date.advance(3,'day').getRelative("day","year"),
                "day_of_year"))\
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))\
            .filter(ee.Filter.eq('orbitProperties_pass', image.get('orbitProperties_pass')))\
            .filter(ee.Filter.eq('instrumentMode', image.get('instrumentMode')))\
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation',
                ee.List(image.get('transmitterReceiverPolarisation')).get(-1)))\
            .filter(ee.Filter.Or(ee.Filter.eq('relativeOrbitNumber_stop', image.get('relativeOrbitNumber_stop')),
                ee.Filter.eq('relativeOrbitNumber_stop', image.get('relativeOrbitNumber_start'))))

    ndrCol = refCol.map(lambda ref: cleanIsolatedPixel(ndr(spkFilter(image,FILTER, KERNEL_SIZE),spkFilter(ref,FILTER, KERNEL_SIZE))))
    flood_merge = ndrCol.mode().rename(["NDR"])
    return flood_merge.selfMask().updateMask(handTh).toUint8()

  elif refMode == "BEFORE":
    refIm =  ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')\
            .filterBounds(image.geometry().centroid())\
            .filterDate('2014-01-01', date.format("yyyy-MM-dd"))\
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))\
            .filter(ee.Filter.eq('orbitProperties_pass', image.get('orbitProperties_pass')))\
            .filter(ee.Filter.eq('instrumentMode', image.get('instrumentMode')))\
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation',
                ee.List(image.get('transmitterReceiverPolarisation')).get(-1)))\
            .filter(ee.Filter.Or(ee.Filter.eq('relativeOrbitNumber_stop', image.get('relativeOrbitNumber_stop')),
                ee.Filter.eq('relativeOrbitNumber_stop', image.get('relativeOrbitNumber_start'))))\
            .sort("system:time_start", False).first()
    ndrIm =  cleanIsolatedPixel(ndr(spkFilter(image,FILTER, KERNEL_SIZE),spkFilter(refIm,FILTER, KERNEL_SIZE))).rename(["NDR"])
    return ndrIm.selfMask().updateMask(handTh).toUint8()

  elif refMode == "DRY":
    refIm =  ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')\
            .filterBounds(image.geometry().centroid())\
            .filterDate('2014-01-01', date.format("yyyy-MM-dd"))\
            .filter(ee.Filter.calendarRange(9,10,"month"))\
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))\
            .filter(ee.Filter.eq('orbitProperties_pass', image.get('orbitProperties_pass')))\
            .filter(ee.Filter.eq('instrumentMode', image.get('instrumentMode')))\
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation',
                ee.List(image.get('transmitterReceiverPolarisation')).get(-1)))\
            .filter(ee.Filter.Or(ee.Filter.eq('relativeOrbitNumber_stop', image.get('relativeOrbitNumber_stop')),
                ee.Filter.eq('relativeOrbitNumber_stop', image.get('relativeOrbitNumber_start'))))\
            .sort("system:time_start", False).first()
    ndrIm =  cleanIsolatedPixel(ndr(spkFilter(image,FILTER, KERNEL_SIZE),spkFilter(refIm,FILTER, KERNEL_SIZE))).rename(["NDR"])
    return ndrIm.selfMask().updateMask(handTh).toUint8()

  else :
    raise ValueError(f"'{refMode}' reference unknown, must be one of 'MULTI', 'BEFORE', or 'DRY'")

##List of available images for the periode


In [ ]:
#@title Retrieve list of available images from earth engine
#Get the country boundary
boundary = ee.FeatureCollection("FAO/GAUL/2015/level0").filter(ee.Filter.eq("ADM0_NAME", COUNTRY)).first()

#Find all avalable sentinel1 images in IW mode for the periode of detection within the boundary
collection = ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT').filterDate(START_DATE, END_DATE)\
                  .filterBounds(boundary.geometry())\
                  .filter(ee.Filter.eq('orbitProperties_pass', ORBIT_PASS))\
                  .filter(ee.Filter.eq('instrumentMode', 'IW'))

list_of_images = collection.aggregate_array('system:index').getInfo()


In [ ]:
print(list_of_images)

---

## Perform analysys and export to ee asset


In [ ]:
#set the name of the image collection asset
colId = "/".join([ASSET_PATH, EVENT])

In [ ]:
import time
from tqdm.auto import tqdm

try:
  asset = ee.data.createAsset({"type":"ImageCollection"},colId)
  time.sleep(5)
except ee.EEException:
  print(f"unable to create asset : {colId}")

print("\n")

taskList = []
for imageId in tqdm(list_of_images, desc="Submit tasks"):
  image = ee.Image('COPERNICUS/S1_GRD_FLOAT/'+imageId).select(POLARISATION)
  ndrIm = ndr_autoRef(image, REFERENCE)

  # Export the computed image to the Earth Engine image collection asset
  task = ee.batch.Export.image.toAsset(**{
    'image': ndrIm.clip(boundary.geometry()),
    'description': imageId,
    'assetId': "/".join([colId,imageId]),
    'scale': SCALE,
    'maxPixels': 1e10,
    'region': image.geometry().getInfo()['coordinates']
  })


  taskList.append(task)
  taskList[-1].start()


Folow tasks progression on
[Task manager](https://code.earthengine.google.com/tasks)

##Display results and export to google drive for download


In [ ]:
import geemap

Map = geemap.Map()
Map.centerObject(boundary, 8)
vizparam = {
    "min": 0,
    "max": 1,
    "palette" : ["00FFFF"]
}
Map.add_basemap("SATELLITE")

image = ee.ImageCollection(colId).max().clip(boundary)

Map.addLayer(image, vizparam, "flood")

Map


In [ ]:
#Export images to drive for download
import time
from tqdm import tqdm
list_output_images = ee.ImageCollection(colId).aggregate_array('system:index').getInfo()

for image_id in tqdm(list_output_images):
  image = ee.Image("/".join([colId,image_id])).clip(boundary)

  drive_export = ee.batch.Export.image.toDrive(**{
    'image': image,
    'description': image_id,
    'folder': EVENT,
    'scale': SCALE,
    'maxPixels': 1e10,
    'region': image.geometry()
  })

  drive_export.start()
  time.sleep(1)

Images are exported to a google drive folder with same name as the EVENT parameter

In [ ]:
drive_export = ee.batch.Export.image.toDrive(**{
    'image': ee.ImageCollection(colId).max().toUint8(),
    'description': EVENT+'_mosaic',
    'folder': EVENT+'_mosaic',
    'scale': SCALE,
    'maxPixels': 1e10,
    'region': boundary.geometry()
  })
drive_export.start()

# References
[1]  J. S. Lee, “Digital image enhancement and noise filtering by use of local statistics,”
    IEEE Pattern Anal. Machine Intell., vol. PAMI-2, pp. 165–168, Mar. 1980. <br>
[2]  A. Lopes, R. Touzi, and E. Nezry, “Adaptative speckle filters and scene heterogeneity,
    IEEE Trans. Geosci. Remote Sensing, vol. 28, pp. 992–1000, Nov. 1990 <br>
[3]  Lopes, A.; Nezry, E.; Touzi, R.; Laur, H.  Maximum a posteriori speckle filtering and first204order texture models in SAR images.
    10th annual international symposium on geoscience205and remote sensing. Ieee, 1990, pp. 2409–2412.<br>
[4] J.-S. Lee, M.R. Grunes, G. De Grandi. Polarimetric SAR speckle filtering and its implication for classification
    IEEE Trans. Geosci. Remote Sens., 37 (5) (1999), pp. 2363-2373.<br>
[5] Lee, J.-S.; Wen, J.-H.; Ainsworth, T.L.; Chen, K.-S.; Chen, A.J. Improved sigma filter for speckle filtering of SAR imagery.
    IEEE Trans. Geosci. Remote Sens. 2009, 47, 202–213.<br>
[6] Mullissa, A., Vollrath, A., Odongo-Braun, C., Slagter, B., Balling, J., Gou, Y., Gorelick, N., Reiche, J., 2021.
    Sentinel-1 SAR Backscatter Analysis Ready Data Preparation in Google Earth Engine. Remote Sensing 13, 1954.
    https://doi.org/10.3390/rs13101954



